# [1174. Immediate Food Delivery II](https://leetcode.com/problems/immediate-food-delivery-ii/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Delivery

<pre>+-----------------------------+---------+
| Column Name                 | Type    |
+-----------------------------+---------+
| delivery_id                 | int     |
| customer_id                 | int     |
| order_date                  | date    |
| customer_pref_delivery_date | date    |
+-----------------------------+---------+</pre>
delivery_id is the column of unique values of this table.
The table holds information about food delivery to customers that make orders at some date and specify a preferred delivery date (on the same order date or after it).
 

If the customer's preferred delivery date is the same as the order date, then the order is called immediate; otherwise, it is called scheduled.

The first order of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has precisely one first order.

Write a solution to find the percentage of immediate orders in the first orders of all customers, rounded to 2 decimal places.

The result format is in the following example.

 

Example 1:

Input: 
Delivery table:
<pre>+-------------+-------------+------------+-----------------------------+
| delivery_id | customer_id | order_date | customer_pref_delivery_date |
+-------------+-------------+------------+-----------------------------+
| 1           | 1           | 2019-08-01 | 2019-08-02                  |
| 2           | 2           | 2019-08-02 | 2019-08-02                  |
| 3           | 1           | 2019-08-11 | 2019-08-12                  |
| 4           | 3           | 2019-08-24 | 2019-08-24                  |
| 5           | 3           | 2019-08-21 | 2019-08-22                  |
| 6           | 2           | 2019-08-11 | 2019-08-13                  |
| 7           | 4           | 2019-08-09 | 2019-08-09                  |
+-------------+-------------+------------+-----------------------------+</pre>
Output: 
<pre>+----------------------+
| immediate_percentage |
+----------------------+
| 50.00                |
+----------------------+</pre>
Explanation: 
The customer id 1 has a first order with delivery id 1 and it is scheduled.
The customer id 2 has a first order with delivery id 2 and it is immediate.
The customer id 3 has a first order with delivery id 5 and it is scheduled.
The customer id 4 has a first order with delivery id 7 and it is immediate.
Hence, half the customers have immediate first orders.

In [1]:
#Pandas schema

import pandas as pd

data = [[1, 1, '2019-08-01', '2019-08-02'], [2, 2, '2019-08-02', '2019-08-02'], [3, 1, '2019-08-11', '2019-08-12'],
        [4, 3, '2019-08-24', '2019-08-24'], [5, 3, '2019-08-21', '2019-08-22'], [6, 2, '2019-08-11', '2019-08-13'],
        [7, 4, '2019-08-09', '2019-08-09']]
delivery = pd.DataFrame(data,
                        columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype(
    {'delivery_id': 'Int64', 'customer_id': 'Int64', 'order_date': 'datetime64[ns]',
     'customer_pref_delivery_date': 'datetime64[ns]'})

In [2]:
# to spark dataframe

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

delivery_df = spark.createDataFrame(delivery)
delivery_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/09 15:01:31 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/09 15:01:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 15:01:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----------+-----------+-------------------+---------------------------+
|delivery_id|customer_id|order_date         |customer_pref_delivery_date|
+-----------+-----------+-------------------+---------------------------+
|1          |1          |2019-08-01 00:00:00|2019-08-02 00:00:00        |
|2          |2          |2019-08-02 00:00:00|2019-08-02 00:00:00        |
|3          |1          |2019-08-11 00:00:00|2019-08-12 00:00:00        |
|4          |3          |2019-08-24 00:00:00|2019-08-24 00:00:00        |
|5          |3          |2019-08-21 00:00:00|2019-08-22 00:00:00        |
|6          |2          |2019-08-11 00:00:00|2019-08-13 00:00:00        |
|7          |4          |2019-08-09 00:00:00|2019-08-09 00:00:00        |
+-----------+-----------+-------------------+---------------------------+


In [3]:
# solving in spark dataframe
from pyspark.sql import Window
import pyspark.sql.functions as F

delivery_df \
    .withColumn('rn', F.row_number().over(Window.partitionBy('customer_id').orderBy('order_date'))) \
    .filter(F.col('rn') == 1) \
    .agg(
    F.round(100*
        F.sum(F.when(F.col('order_date') == F.col('customer_pref_delivery_date'), 1).otherwise(0)) / F.count('*'),
        2) \
        .alias('immediate_percentage')
) \
    .show()

+--------------------+
|immediate_percentage|
+--------------------+
|                50.0|
+--------------------+


In [4]:
# solving in spark SQL

delivery_df.createOrReplaceTempView('delivery')

spark.sql('''
with tbl as (
select *, row_number() over (partition by customer_id order by order_date) as rn
from delivery
)
select 
    round(
    100*sum(case when order_date=customer_pref_delivery_date then 1 else 0 end)/count(*),
    2) as immediate_percentage
from tbl
where rn = 1;
''').show()

+--------------------+
|immediate_percentage|
+--------------------+
|                50.0|
+--------------------+


In [5]:
spark.stop()